<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pickle
import pandas as pd

# 🔹 Datei laden
file_path = "features_with_labels.pkl"
with open(file_path, "rb") as pkl_file:
    features_with_labels_dic = pickle.load(pkl_file)

# 🔹 Nur relevante Features auswählen
selected_features = ["COM_to_ground", "knee_angles_right", "knee_angles_left"]
frames = list(features_with_labels_dic["COM_to_ground"].keys())
labels = [features_with_labels_dic["Schwung_labels"].get(frame, 0) for frame in frames]

# 🔹 DataFrame mit den ausgewählten Features erstellen
data = {"Frame": frames, "Label": labels}
for feature in selected_features:
    data[feature] = [features_with_labels_dic[feature][frame] for frame in frames]

df = pd.DataFrame(data)

# 🔹 DataFrame anzeigen
df.head()

,Frame,Label,COM_to_ground,knee_angles_right,knee_angles_left
0,0,1,0.835161,40.842270,35.111111
1,1,1,0.835312,41.973206,34.066200
2,2,1,0.825844,46.869869,37.080631
3,3,1,0.823345,47.376663,39.021801
4,4,1,0.813417,49.918045,41.960918


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 🔹 LSTM Input vorbereiten
selected_features = ["COM_to_ground", "knee_angles_right", "knee_angles_left"]
X = df[selected_features].values.reshape(1, -1, len(selected_features))  # Shape: (Batch=1, Time, Features)
Y = df["Label"].values.reshape(1, -1)  # Shape: (Batch=1, Time)

# In Torch-Tensor umwandeln
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.long)

# 🔹 LSTM-Modell definieren
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        return out

# 🔹 Modell initialisieren
input_size = len(selected_features)
hidden_size = 64
num_layers = 2
output_size = 3  # 0 = Kein Schwung, 1 = Linksschwung, 2 = Rechtsschwung

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 🔹 Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔹 Training
num_epochs = 50
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(0), Y_train.squeeze(0))
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training abgeschlossen! 🚀")


Epoch 0, Loss: 1.058222770690918
Epoch 10, Loss: 0.6797165274620056
Epoch 20, Loss: 0.4310837984085083
Epoch 30, Loss: 0.2641715109348297
Epoch 40, Loss: 0.16019731760025024
Training abgeschlossen! 🚀


In [8]:
with torch.no_grad():  # Keine Gradientenberechnung nötig
    predictions = model(X_train)
    predicted_classes = torch.argmax(predictions, dim=2)  # Klasse mit höchster Wahrscheinlichkeit
print(predicted_classes)


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [9]:
correct = (predicted_classes.squeeze(0) == Y_train.squeeze(0)).sum().item()
accuracy = correct / len(Y_train.squeeze(0))
print(f"Modellgenauigkeit: {accuracy:.2f}")


Modellgenauigkeit: 0.98
